In [2]:
import torch # Verify that MPS is availabe on M1 Mac
print(torch.__version__)
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

2.0.0.dev20230226
tensor([1.], device='mps:0')


In [13]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cpu")

In [14]:
%%timeit
x = T.randn(1000, 1000, device=device)
y = T.randn(1000, 1000, device=device)
z = T.matmul(x, y)

89.4 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
# Import a model
from torchvision import models
model = models.resnet18(weights=True)
model.eval()
# Get number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params:,}")

/Users/theodorjonsson/opt/anaconda3/envs/pt/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/theodorjonsson/opt/anaconda3/envs/pt/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/theodorjonsson/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [27]:
model = model.to(device)
sample_input = T.randn(1, 3, 224, 224, device=device)
compiled_model = T.compile(model)
T._dynamo.config.suppress_errors = True

In [28]:
%%timeit
with T.no_grad():
    out = compiled_model(sample_input)

[2023-02-26 15:48:38,176] torch._dynamo.convert_frame: [ERROR] WON'T CONVERT forward /Users/theodorjonsson/opt/anaconda3/envs/pt/lib/python3.9/site-packages/torchvision/models/resnet.py line 284 
due to: 
Traceback (most recent call last):
  File "/Users/theodorjonsson/opt/anaconda3/envs/pt/lib/python3.9/site-packages/torch/_inductor/codecache.py", line 576, in load
    from filelock import FileLock
ModuleNotFoundError: No module named 'filelock'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/theodorjonsson/opt/anaconda3/envs/pt/lib/python3.9/site-packages/torch/_dynamo/output_graph.py", line 713, in call_user_compiler
    raise BackendCompilerFailed(self.compiler_fn, e) from e
torch._dynamo.exc.BackendCompilerFailed: debug_wrapper raised ModuleNotFoundError: No module named 'filelock'

Set torch._dynamo.config.verbose=True for more information


[2023-02-26 15:48:39,622] torch._dynamo.convert_frame: [ERROR] WON'

The slowest run took 4.79 times longer than the fastest. This could mean that an intermediate result is being cached.
180 ms ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit
with T.no_grad():
    out = model(sample_input)

86.2 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
